# Imports and Installs

In [ ]:
%pip install --upgrade --quiet langchain-google-genai
%pip install --upgrade --quiet  wikipedia
%pip install -U langchain-community
%pip install -U langgraph

import getpass
import os

# to set up agent
from langchain_google_genai import GoogleGenerativeAI, ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

# for react & tools
from langchain.agents import AgentExecutor, initialize_agent, create_react_agent, Tool
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import AgentType
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

# Setting Up OpenAPI

# Setting up Google GenAI
Source for Google GenAI Embeddings in LangChain

https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/

Source to Embedding Models

https://python.langchain.com/docs/integrations/text_embedding/

In [ ]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")

chatbot = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Initial Test of Model

List of Google AI Models
https://ai.google.dev/gemini-api/docs/models/gemini


In [ ]:
# create a message
msg = HumanMessage(content=input("Hello. How can I help?\n"), name="Saranya")

# message list
messages = [msg]

chatbot.invoke(messages)


Hello. How can I help?
what color is the sky?


'Blue (most of the time).  It can also appear other colors like gray, white, orange, red, or even violet depending on the time of day and weather conditions.\n'

# trying to loop to continuously ask

In [ ]:
while True:
  #print("starting here")
  content1 = input("Hello. How can I help?\n")
  msg = HumanMessage(content=content1, name="User")


  if content1.lower() == "bye":
    print("Bye!")
    break

  messages = [msg]
  output = chatbot.invoke(messages)
  print(output)

Hello. How can I help?
What number comes after 2?
3

Hello. How can I help?
What is the capital of California?
Sacramento

Hello. How can I help?
Bye
Bye!


# Zero-Shot Prompting

In [ ]:
system_msg = SystemMessage(content="Translate what the user says into Tamil.")

while True:
  content1 = input("Hello. How can I help?\n")
  human_msg = HumanMessage(content=content1, name="User")

  if content1.lower() == "bye":
    print("Bye!")
    break

  messages = [system_msg, human_msg]
  output = chatbot.invoke(messages)
  print(output)

Hello. How can I help?
Hello
வணக்கம் (Vaṇakkam)

Hello. How can I help?
Cat
பூனை (Pūnai)

Hello. How can I help?
School
பள்ளி (paḷḷi)

Hello. How can I help?
bye
Bye!


# Few-Shot Prompting
Few shot prompting uses input/output examples that allows for the output to be formatted as demonstrated by the system.

In [ ]:
system_msg = SystemMessage(content="""
Translate what the user says into Tamil.
Examples:
Hello // வணக்கம்
Cat // பூனை
School // பள்ளி
""")

while True:
  content1 = input("Hello. How can I help?\n")
  human_msg = HumanMessage(content=content1, name="User")

  if content1.lower() == "bye":
    print("Bye!")
    break

  messages = [system_msg, human_msg]
  output = chatbot.invoke(messages)
  print(output)

Hello. How can I help?
bag
பை

Hello. How can I help?
chair
நாற்காலி

Hello. How can I help?
bed
படுக்கை

Hello. How can I help?
bye
Bye!


# Few-Shot Prompting Use Cases

**Providing medical diagnosis in a predefined format established from examples given to the chatbot**

In [ ]:
system_msg = SystemMessage(content="""
You are a doctor providing medical diagnoses to patients. Find the diagnosis and provide an explanation.
Example:

Input:
runny nose, sneezing, congestion
Output:
Diagnosis: Common cold
Explanation: The combination of fever, cough, and fatigue symptoms align with the symptoms of a common cold.
Treatment: Cough medicine, over-the-counter products

Input:
burning pain in chest and throat, worse when lying down
Output:
Diagnosis: Heartburn
Explanation: Buring chest, neck, and throat discomfort that worsens when lying down are symptoms of heartburn.
Treatment: Omeprazole or ranitidine. Avoid fats, chocolate, and citrus.
""")

while True:
  content1 = input("Hello. How can I help?\n")
  human_msg = HumanMessage(content=content1, name="User")

  if content1.lower() == "bye":
    print("Bye!")
    break

  messages = [system_msg, human_msg]
  output = chatbot.invoke(messages)
  print(output)

Hello. How can I help?
nausea, sensitivity to light, head throbbing
Diagnosis: Migraine
Explanation: Nausea, sensitivity to light (photophobia), and throbbing head pain are classic symptoms of a migraine. 

Treatment: Rest in a dark, quiet room. Over-the-counter pain relievers such as ibuprofen or naproxen can help.  If over-the-counter medications are not effective, a physician may prescribe specific migraine medications (triptans or CGRP inhibitors).  It is important to discuss your symptoms and treatment options with a doctor for proper diagnosis and management.

Hello. How can I help?
nausea, confusion, dizziness
Diagnosis: Possible Dehydration, Inner Ear Infection, or other causes.

Explanation: Nausea, confusion, and dizziness can be symptoms of various conditions, including dehydration, inner ear infections (labyrinthitis or vestibular neuritis), low blood sugar, or even more serious issues like stroke or transient ischemic attack (TIA).  It's crucial to determine the underlying

# Chain of Thought Prompting

Multistep problems

In [ ]:
system_msg = SystemMessage(content="""Explain step-by-step. Answer the inputs with the following thought process.
Input: If I have 3 bags of 5 marbles, and I give 2 marbles to 4 friends,
how many marbles do I have left?
Output: You have 3 x 5 = 15 marbles. You give away 2 x 4 = 8 marbles. 15 - 8 = 7
marbles.
""")

while True:
  content1 = input("Hello. How can I help?\n")
  human_msg = HumanMessage(content=content1, name="User")

  if content1.lower() == "bye":
    print("Bye!")
    break

  messages = [system_msg, human_msg]
  output = chatbot.invoke(messages)
  print(output)

# ReAct Prompting Use Case

react: try to get the model to to the reasoning before getting the result.

ReAct Implementation Link: https://www.youtube.com/watch?v=Eug2clsLtFs&t=703s

Wikipedia tool: https://python.langchain.com/docs/integrations/tools/wikipedia/

Initializing Agent Parameters Link: https://python.langchain.com/api_reference/langchain/agents/langchain.agents.initialize.initialize_agent.html

In [ ]:
system_msg = SystemMessage(content="""
Respond using the ReAct technique of Thought, Action, Observation.
""")

wikipedia = WikipediaAPIWrapper()

tools = [
    Tool(
        name="Wikipedia Search",
        func=wikipedia.run,
        description="useful for searching Wikipedia articles."
    )
]

react = initialize_agent(tools, chatbot, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = input("Ask a question: \n")

react.run(question)

'''while True:
  content1 = input("Hello. How can I help?\n")
  human_msg = HumanMessage(content=content1, name="User")

  if content1.lower() == "bye":
    print("Bye!")
    break

  messages = [system_msg, human_msg]
  output = chatbot.invoke(messages)
  print(output)
  '''

Ask a question: 
Who made the first personal computer?


> Entering new AgentExecutor chain...
I need to find out who made the first personal computer. I will use Wikipedia to search for this information.
Action: Wikipedia Search
Action Input: "first personal computer"
Observation: Page: Personal computer
Summary: A personal computer, often referred to as a PC or simply computer, is a computer designed for individual use. It is typically used for tasks such as word processing, internet browsing, email, multimedia playback, and gaming. Personal computers are intended to be operated directly by an end user, rather than by a computer expert or technician. Unlike large, costly minicomputers and mainframes, time-sharing by many people at the same time is not used with personal computers. The term home computer has also been used, primarily in the late 1970s and 1980s. The advent of personal computers and the concurrent Digital Revolution have significantly affected the lives of people.
Inst

'while True:\n  content1 = input("Hello. How can I help?\n")\n  human_msg = HumanMessage(content=content1, name="User")\n\n  if content1.lower() == "bye":\n    print("Bye!")\n    break\n\n  messages = [system_msg, human_msg]\n  output = chatbot.invoke(messages)\n  print(output)\n  '

# Calculator

Custom Tool Calling: https://www.youtube.com/watch?v=zCwuAlpQKTM

https://python.langchain.com/docs/how_to/tool_calling/

https://python.langchain.com/docs/integrations/chat/

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

Prebuilt React Agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/#code

In [ ]:
# tools
@tool
def add(a: int, b: int) -> int:
  '''Adds two numbers.'''
  return a + b


@tool
def multiply(a: int, b: int) -> int:
  '''Multiplies two numbers.'''
  return a * b

@tool
def divide(a: int, b:int):
  '''Divides two numbers'''
  return a / b

# using tool calling
custom_tools = [add, multiply, divide]

# using prebuilt react
pre_agent = create_react_agent(chatbot, tools=custom_tools)
inputs = {"messages": [("user", "add 2 and 3")]}
print_stream(pre_agent.stream(inputs, stream_mode="values"))


# '''
# chat_w_tools = chatbot.bind_tools(tools)

# query = input("Ask a question: \n")
# chat_w_tools.invoke(query)


# # using agent
# agent = initialize_agent(tools, chatbot, agent=AgentType.OPENAI_MULTI_FUNCTIONS, verbose=True)
# agent.run(query)
# '''



================================ Human Message =================================

add 2 and 3
================================== Ai Message ==================================
Tool Calls:
  add (3b44ffe3-b971-460c-af1f-3b9ce8169efc)
 Call ID: 3b44ffe3-b971-460c-af1f-3b9ce8169efc
  Args:
    a: 2.0
    b: 3.0
================================= Tool Message =================================
Name: add

5
================================== Ai Message ==================================

The sum of 2 and 3 is 5.


# Unit Conversion Tool

In [ ]:
@tool
def length_conversion(value: float, from_unit: str, to_unit: str):
  '''Used to convert length measurements.'''
  units = {
      "mm": 0.001,
      "cm": 0.01,
      "m": 1,
      "km": 1000,
      "in": 0.0254,
      "ft": 0.3048
  }

  # convert to meters
  meters = value * units[from_unit]

  # convert to desired unit
  result = meters / units[to_unit]

  return result


@tool
def weight_conversion(value: float, from_unit: str, to_unit: str):
  '''Used to convert weight measurements.'''
  units = {
      "kg": 0.453592,
      "g": 453.592,
      "lbs": 1,
      "ounces": 16,
  }

  # convert to lbs
  pounds_val = value / units[from_unit]

  # convert from lbs to specified value
  converted_val = pounds_val * units[to_unit]

  return converted_val

# using tool calling
custom_tools = [length_conversion, weight_conversion]

# using prebuilt react
pre_agent = create_react_agent(chatbot, tools=custom_tools)
inputs = {"messages": [("user", "convert 2 lbs to kg")]}
print_stream(pre_agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

convert 2 lbs to kg
================================== Ai Message ==================================
Tool Calls:
  weight_conversion (4f6dedf4-34fb-47dc-bae2-f241433a96ad)
 Call ID: 4f6dedf4-34fb-47dc-bae2-f241433a96ad
  Args:
    value: 2.0
    from_unit: lbs
    to_unit: kg
================================= Tool Message =================================
Name: weight_conversion

0.907184
================================== Ai Message ==================================

2 lbs is equal to 0.907184 kg.
